In [2]:
from tensorflow import keras

### Layers

In [1]:
import re
from requests import get

from selenium.webdriver import Chrome
from json import load,dump

In [2]:
bot = Chrome()

In [3]:
base_url = 'https://keras.io/api/layers'

In [4]:
bot.get(base_url)

In [5]:
groups = [ group.text.lower().replace("-","").replace(" ","-") for group in bot.find_elements_by_tag_name("h3")[5:]]

In [12]:
elem = None
layers = {}
REQUIRED = 'required'

def get_render_list(arg,val)->str:
    if val.istitle():
        val = eval(val)
    if type(val) == bool:
        return "bool"
    return arg.split("_")[-1]

def get_render_type(arg,val:str)->str:
    if val.istitle():
        val = eval(val)
        
    if type(val) == bool:
        return "list"
    elif type(val) == str or val == None:
        if arg.split("_")[-1] in ['activation','initializer','regularizer','constraint']:
            return "list"
        return 'text'
    else:
        return "text"

def get_type(val:str):
    return re.sub("([<>''])|(class )","",str(type(val)))
    
def get_args(code:str)->dict:
    args = [ 
        a0[:-1].split("=") 
            if 
        a0 
            else 
        (a1[:-1],"REQUIRED") 
        for a0,a1 in  re.findall("(\w+=[a-zA-Z0-9\(\),. ]+)|(\w+,)",code) 
    ]

    return {
        arg:{
            "value":val.__str__(),
            "type":get_type(val),
            "render":get_render_type(arg,val),
            "options":get_render_list(arg,val)
        }
        for arg,val in args
    }

for group in groups:
    bot.get(base_url)
    elem = bot.find_element_by_id(group)
    elem.find_element_by_tag_name("a").click()
    
    group_url = bot.current_url
    group_name = bot.find_element_by_id(group).text
    
    layer_urls = [ (a.text,a.get_property("href")) for a in bot.find_element_by_tag_name("ul").find_elements_by_tag_name("a")]
    group_layers = []
    for name,url in layer_urls:
        bot.get(url)
        layer_name = re.sub("( layer)|( object)|( layers)","",name)
        code = bot.find_element_by_tag_name("pre")
        group_layers.append(dict(
            name=layer_name,
            type=dict(
                name=layer_name,
                _class="layers"
            ),
            args=get_args(code.text),
            doc=url
        ))
        
    layers[group] = dict(
        name=group_name,
        layers=group_layers,
        doc=group_url
    )

In [15]:
layers.update({
    "build-layers":{
        "name":"Build Tools",
        "layers":[
            {
                "name":"Model",
                "type":{
                    "name":"Model",
                    "_class":"models"
                },
                "args":{

                }
            },
            {
                "name":"Compile",
                "type":{
                    "name":"Compile",
                    "_class":"models"
                },
                "args":{
                    "optmizer":{
                        "value":"rmsprop",
                        "type":"str",
                        "render":"list",
                        "options": "optimizer"
                    },
                    "loss":{
                        "value":"None",
                        "type":"str",
                        "render":"list",
                        "options": "loss"
                    },
                    "metrics":{
                        "value":[],
                        "type":"str",
                        "render":"checkbox",
                        "options": "metrics"
                    }
                }
            },
            {
                "name":"Train",
                "type":{
                    "name":"Train",
                    "_class":"models"
                },
                "args":{
                    "batch_size":{
                        "value": "8",
                        "type": "int",
                        "render": "text",
                        "options": "batch_size",
                    },
                    "epochs":{
                        "value": "1",
                        "type": "int",
                        "render": "text",
                        "options": "batch_size",
                    },
                }
            }
        ]
    }
})

In [17]:
out = layers.__repr__()
out = re.sub("\d+: \{[a-zA-Z0-9 ;:,]\},","",out)
out = re.sub("None,","None",out)

with open("../html/src/data/layers.js","w+") as file:
    file.write(f"""
import datasets from './datasets';
import optimizers from './optimizers';
import callbaacks from './callbacks';

const layers = {out}

export default layers;
""")


### Callbacks

In [423]:
code="""tf.keras.layers.DepthwiseConv2D(
    kernel_size,
    strides=(1, 1),
    padding="valid",
    depth_multiplier=1,
    data_format=None,
    dilation_rate=(1, 1),
    activation=None,
    use_bias=True,
    depthwise_initializer="glorot_uniform",
    bias_initializer="zeros",
    depthwise_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    depthwise_constraint=None,
    bias_constraint=None,
    **kwargs
)"""

In [424]:
re.findall("(\w+=[a-zA-Z0-9\(\), ]+)|(\w+,)",code)

[('', 'kernel_size,'),
 ('strides=(1, 1),', ''),
 ('depth_multiplier=1,', ''),
 ('data_format=None,', ''),
 ('dilation_rate=(1, 1),', ''),
 ('activation=None,', ''),
 ('use_bias=True,', ''),
 ('depthwise_regularizer=None,', ''),
 ('bias_regularizer=None,', ''),
 ('activity_regularizer=None,', ''),
 ('depthwise_constraint=None,', ''),
 ('bias_constraint=None,', '')]

In [18]:
import re
from requests import get

In [5]:

def _get_render_list(arg,val)->str:
    try:
        if val.istitle():
            val = eval(val)
    except:
        pass
    
    if type(val) == bool:
        return "bool"
    return arg.split("_")[-1]

def _get_render_type(arg,val:str)->str:
    try:
        if val.istitle():
            val = eval(val)
    except:
        pass
            
    if type(val) == bool:
        return "list"
    elif type(val) == str or val == None:
        if arg.split("_")[-1] in ['activation','initializer','regularizer','constraint']:
            return "list"
        return 'text'
    else:
        return "text"

In [21]:
callback_urls = re.findall("""/api/callbacks/\w+""",get("https://keras.io/api/callbacks/").text)
config = []

for url in callback_urls:
    args = re.findall(
            """tf.keras.callbacks.\w+\([a-zA-Z0-9 \t\n,;:="\n_\*]+""",
            re.sub("""(<[a-zA-Z-=0-9\" ,./!:?@;&+\']*>)|(</[a-zA-Z=0-9\" ]*>)|(\n)|(&quot;)""",'',get(f"https://keras.io{url}").text)
        )
    args = args[0] if len(args) else None
    cls, args = args.split("(") if args else ( None,None )
    
    if cls:
        args = [
                arg.strip().split("=") if "=" in arg else (arg.strip().replace(",",''),"REQUIRED") for arg in args.split(", ") if 'kwargs' not in arg
            ] if args else []
        config.append({
            "name":cls.split(".")[-1],
            "type":{
                "name":cls.split(".")[-1],
                "_class":"callbacks"
            },
            "args":{
                arg:{
                    "value":val.__str__(),
                    "type":re.sub("([<>''])|(class )","",str(type(val))),
                    "render":_get_render_type(arg,val),
                    "options":_get_render_list(arg,val)
                }
                for arg,val in args
            }
        })        

In [22]:
with open("../html/src/data/callbacks.js","w+") as file:
    file.write(f"""
const callbacks = {"{"}
    "name":"Callbacks",
    "layers":{config.__repr__()}
{"}"}
export default callbacks;
""")

### Optmizers

In [6]:
urls = re.findall("""/api/optimizers/\w+""",get("https://keras.io/api/optimizers/").text)
config = []

    
for url in urls:
    args = re.findall(
            """tf.keras.optimizers.\w+\([a-zA-Z0-9 \t\n,;:="\n_\*\.-]+""",
            re.sub("""(<[a-zA-Z-=0-9\" ,./!:?@;&+\']*>)|(</[a-zA-Z=0-9\" ]*>)|(\n)|(&quot;)""",'',get(f"https://keras.io{url}").text)
        )
    args = args[0] if len(args) else None
    callback, args = args.split("(") if args else ( None,None )
    
    if callback:
        args = [
                arg.strip().split("=") if "=" in arg else (arg.strip().replace(",",''),"REQUIRED") for arg in args.split(", ") if 'kwargs' not in arg
            ]
        config.append({
            "name":callback.split(".")[-1],
            "type":{
                "name":callback.split(".")[-1],
                "_class":"optimizers"
            },
            "args":{
                arg:{
                    "value":val.__str__(),
                    "type":re.sub("([<>''])|(class )","",str(type(val))),
                    "render":_get_render_type(arg,val),
                    "options":_get_render_list(arg,val)
                }
                for arg,val in args
            }
        })

In [7]:
with open("../html/src/data/optimizers.js","w+") as file:
    file.write(f"""
const optimizers = {"{"}
    "name":"Optimizers",
    "layers":{config.__repr__()}
{"}"}
export default optimizers;
""")

### Options

In [445]:
options = dict(
    activation = [
        'None',
        'elu',
        'exponential',
        'gelu',
        'get',
        'hard_sigmoid',
        'linear',
        'relu',
        'selu',
        'serialize',
        'sigmoid',
        'softmax',
        'softplus',
        'softsign',
        'swish',
        'tanh'
    ],

    constraint = [
        'None',
        'max_norm',
        'min_max_norm',
        'non_neg',
        'radial_constraint',
        'serialize',
        'unit_norm'
    ],

    regularizer = [
        'None',
        'l1',
        'l1_l2',
        'l2',
    ],

    initializer = [
        'None',
        'glorot_normal',
        'glorot_uniform',
        'he_normal',
        'he_uniform',
        'identity',
        'lecun_normal',
        'lecun_uniform',
        'ones',
        'orthogonal',
        'random_normal',
        'random_uniform',
        'serialize',
        'truncated_normal',
        'variance_scaling',
        'zeros'
    ],

    metrics = [
        'None',
        'binary_accuracy',
        'binary_crossentropy',
        'categorical_accuracy',
        'categorical_crossentropy',
        'deserialize',
        'get',
        'hinge',
        'kl_divergence',
        'kld',
        'kullback_leibler_divergence',
        'log_cosh',
        'logcosh',
        'mae',
        'mape',
        'mean_absolute_error',
        'mean_absolute_percentage_error',
        'mean_squared_error',
        'mean_squared_logarithmic_error',
        'mse',
        'msle',
        'poisson',
        'serialize',
        'sparse_categorical_accuracy',
        'sparse_categorical_crossentropy',
        'sparse_top_k_categorical_accuracy',
        'squared_hinge',
        'top_k_categorical_accuracy'
    ],

    loss = [
        'None',
        'binary_crossentropy',
        'categorical_crossentropy',
        'categorical_hinge',
        'cosine_similarity',
        'deserialize',
        'get',
        'hinge',
        'huber',
        'kl_divergence',
        'kld',
        'kullback_leibler_divergence',
        'log_cosh',
        'logcosh',
        'mae',
        'mape',
        'mean_absolute_error',
        'mean_absolute_percentage_error',
        'mean_squared_error',
        'mean_squared_logarithmic_error',
        'mse',
        'msle',
        'poisson',
        'serialize',
        'sparse_categorical_crossentropy',
        'squared_hinge'
    ],

    optimizer = [
        'None',
        'adadelta',
        'adagrad',
        'adam',
        'adamax',
        'ftrl',
        'nadam',
        'optimizer',
        'rmsprop',
        'sgd'
    ],
    bool = [
        'None',
        'True',
        'False'
    ],
    metrics = [
     'binary_accuracy',
     'binary_crossentropy',
     'categorical_accuracy',
     'categorical_crossentropy',
     'deserialize',
     'get',
     'hinge',
     'kl_divergence',
     'kld',
     'kullback_leibler_divergence',
     'log_cosh',
     'logcosh',
     'mae',
     'mape',
     'mean_absolute_error',
     'mean_absolute_percentage_error',
     'mean_squared_error',
     'mean_squared_logarithmic_error',
     'mse',
     'msle',
     'poisson',
     'serialize',
     'sparse_categorical_accuracy',
     'sparse_categorical_crossentropy',
     'sparse_top_k_categorical_accuracy',
     'squared_hinge',
     'top_k_categorical_accuracy'
    ]
)

In [446]:
with open("../html/src/data/options.js","w+") as file:
    file.write(f"""
const options = {options.__repr__()}

export default options;
""")